ImportError: cannot import name 'ufunc' from 'numpy' (unknown location)

In [12]:
!which python
!which pip

/Users/serkanmp/anaconda3/bin/python
/Users/serkanmp/anaconda3/bin/pip


In [13]:
!where python
!where pip

/Users/serkanmp/anaconda3/bin/python
/Users/serkanmp/anaconda3/bin/pip
/Library/Frameworks/Python.framework/Versions/3.9/bin/pip


In [14]:
!pip cache purge
!pip install numpy tensorflow

Files removed: 1162


In [23]:

!pip install tensorflow 

In [24]:
import numpy as np
import tensorflow as tf

print("NumPy version:", np.__version__)
print("TensorFlow version:", tf.__version__)

ImportError: cannot import name 'ufunc' from 'numpy' (unknown location)

In [2]:
pip install --upgrade numpy tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 17.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.3/227.3 kB 19.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-datasets 4.9.4 requires toml, which is not installed.
wordcloud 1.9.3 requires matplotlib, which is not installed.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from methods.fgsm import create_adversarial_pattern
from models.mobilenet import load_mobilenet_model, preprocess_image, decode_predictions

def display_image(image, description, model):
    probs = model.predict(image)
    _, label, confidence = decode_predictions(probs)
    plt.figure()
    plt.imshow(image[0] * 0.5 + 0.5)
    plt.title('{} \n {} : {:.2f}% Confidence'.format(description, label, confidence * 100))
    plt.show()

def main():
    # Load model
    model = load_mobilenet_model()

    # Load and preprocess image
    image_path = tf.keras.utils.get_file('YellowLabradorLooking_new.jpg', 'https://storage.googleapis.com/download.tensorflow.org/example_images/YellowLabradorLooking_new.jpg')
    image_raw = tf.io.read_file(image_path)
    image = tf.image.decode_image(image_raw)
    image = preprocess_image(image)

    # Display original image and prediction
    display_image(image, 'Original Image', model)

    # Create adversarial pattern
    prediction = model.predict(image)
    label = tf.one_hot(tf.argmax(prediction[0]), prediction.shape[-1])
    label = tf.reshape(label, (1, prediction.shape[-1]))

    perturbations = create_adversarial_pattern(image, label, model)

    # Use a single epsilon value for the adversarial example
    epsilon = 0.01
    adv_image = image + epsilon * perturbations
    adv_image = tf.clip_by_value(adv_image, -1, 1)
    description = 'Adversarial Image with Epsilon = {:0.3f}'.format(epsilon)
    display_image(adv_image, description, model)

if __name__ == "__main__":
    main()

ImportError: cannot import name 'typing' from 'numpy' (unknown location)

In [8]:
conda update tensorflow


PackageNotInstalledError: Package is not installed in prefix.
  prefix: /Users/serkanmp/anaconda3
  package name: tensorflow



Note: you may need to restart the kernel to use updated packages.


In [1]:
import tensorflow as tf
import numpy as np
from methods.fgsm import create_adversarial_pattern
from models.mobilenet import load_mobilenet_model, preprocess_image, decode_predictions
import tensorflow_datasets as tfds

def evaluate_dataset(dataset, model):
    correct_predictions = 0
    total_predictions = 0

    for image_batch, label_batch in dataset:
        predictions = model.predict(image_batch)
        predicted_labels = np.argmax(predictions, axis=-1)
        correct_predictions += np.sum(predicted_labels == label_batch)
        total_predictions += image_batch.shape[0]

    accuracy = correct_predictions / total_predictions
    return accuracy

def generate_adversarial_dataset(dataset, model, epsilon=0.01):
    adversarial_images = []
    adversarial_labels = []

    for image_batch, label_batch in dataset:
        for image, label in zip(image_batch, label_batch):
            image = tf.expand_dims(image, axis=0)  # Add batch dimension
            prediction = model.predict(image)
            one_hot_label = tf.one_hot(label, prediction.shape[-1])
            one_hot_label = tf.reshape(one_hot_label, (1, -1))  # Ensure it has the correct shape
            perturbations = create_adversarial_pattern(image, one_hot_label, model)
            adv_image = image + epsilon * perturbations
            adv_image = tf.clip_by_value(adv_image, -1, 1)
            adversarial_images.append(adv_image[0])
            adversarial_labels.append(label)

    adversarial_images = np.array(adversarial_images)
    adversarial_labels = np.array(adversarial_labels)
    return tf.data.Dataset.from_tensor_slices((adversarial_images, adversarial_labels)).batch(32)

def preprocess_imagenet_images(images):
    images = tf.image.resize(images, (224, 224))  # Resize to match MobileNet input size
    images = tf.keras.applications.mobilenet.preprocess_input(images)  # Normalize using MobileNet preprocessing
    return images

def main():
    # Load the pre-trained MobileNet model
    model = load_mobilenet_model()

    # Load and preprocess a subset of the ImageNet validation dataset
    imagenet_val, info = tfds.load('imagenet2012', split='validation[:1%]', with_info=True, as_supervised=True)
    imagenet_val = imagenet_val.take(100)  # Take only 100 samples
    imagenet_val = imagenet_val.map(lambda img, label: (preprocess_imagenet_images(img), label)).batch(32)

    # Evaluate the model on the original dataset
    original_accuracy = evaluate_dataset(imagenet_val, model)
    print('Original Accuracy: {:.2f}%'.format(original_accuracy * 100))

    # Generate adversarial examples
    adversarial_test_dataset = generate_adversarial_dataset(imagenet_val, model)

    # Evaluate the model on the adversarial dataset
    adversarial_accuracy = evaluate_dataset(adversarial_test_dataset, model)
    print('Adversarial Accuracy: {:.2f}%'.format(adversarial_accuracy * 100))

if __name__ == "__main__":
    main()

ImportError: dlopen(/Users/serkanmp/anaconda3/lib/python3.10/site-packages/tensorflow/python/platform/_pywrap_cpu_feature_guard.so, 0x0002): Library not loaded: @rpath/libtensorflow_cc.2.dylib
  Referenced from: <39E77809-AF63-396D-A4A9-81818B0B7FF6> /Users/serkanmp/anaconda3/lib/python3.10/site-packages/tensorflow/python/_pywrap_tensorflow_internal.so
  Reason: tried: '/Users/serkanmp/anaconda3/lib/python3.10/site-packages/tensorflow/python/../../_solib_darwin_arm64/_U_S_Stensorflow_Spython_C_Upywrap_Utensorflow_Uinternal.so_Ucclib___Utensorflow/libtensorflow_cc.2.dylib' (no such file), '/Users/serkanmp/anaconda3/lib/python3.10/site-packages/tensorflow/python/_pywrap_tensorflow_internal.so.runfiles/org_tensorflow/_solib_darwin_arm64/_U_S_Stensorflow_Spython_C_Upywrap_Utensorflow_Uinternal.so_Ucclib___Utensorflow/libtensorflow_cc.2.dylib' (no such file), '/Users/serkanmp/anaconda3/lib/python3.10/site-packages/tensorflow/python/../../_solib_darwin_arm64/_Utensorflow/libtensorflow_cc.2.d

In [25]:
import tensorflow as tf
import matplotlib.pyplot as plt
from methods.fgsm import create_adversarial_pattern
from models.mobilenet import load_mobilenet_model, preprocess_image, decode_predictions

def display_images(sample_images, descriptions, model):
    for idx, (original, adversarial) in enumerate(sample_images):
        plt.figure(figsize=(10, 5))

        plt.subplot(1, 2, 1)
        original_probs = model.predict(original)
        _, original_label, original_confidence = decode_predictions(original_probs)
        plt.imshow(original[0] * 0.5 + 0.5)
        plt.title('{} \n {} : {:.2f}% Confidence'.format(descriptions[idx][0], original_label, original_confidence * 100))

        plt.subplot(1, 2, 2)
        adversarial_probs = model.predict(adversarial)
        _, adv_label, adv_confidence = decode_predictions(adversarial_probs)
        plt.imshow(adversarial[0] * 0.5 + 0.5)
        plt.title('{} \n {} : {:.2f}% Confidence'.format(descriptions[idx][1], adv_label, adv_confidence * 100))

        plt.show()

def main():
    method_name = "FGSM"
    model_name = "MobileNet"

    # Load model
    model = load_mobilenet_model()

    # Load CIFAR-10 dataset
    (train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()
    test_images = test_images[:100]  # Use a subset for demonstration purposes
    test_labels = test_labels[:100]

    # Define epsilon
    epsilon = 0.1

    sample_images = []
    descriptions = []

    for i in range(len(test_images)):
        image = test_images[i]
        image = preprocess_image(image)
        
        # Create adversarial image
        prediction = model.predict(image)
        perturbations = create_adversarial_pattern(image, prediction, model)
        adv_image = image + epsilon * perturbations
        adv_image = tf.clip_by_value(adv_image, -1, 1)

        # Store images and descriptions for display
        sample_images.append((image, adv_image))
        descriptions.append(("Original Image", "Adversarial Image"))

    # Display sample images with predictions
    display_images(sample_images, descriptions, model)

if __name__ == "__main__":
    main()